In [ ]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.model_selection import train_test_split
from pennylane.optimize import NesterovMomentumOptimizer

In [ ]:
%run DATA_encode.ipynb
%run Simplified_two_design.ipynb
%run Hardware_efficient_ansatz.ipynb
sep_file4 = read_from_hdf5("Five_qubits_state_seperable.h5","state")
eng_file4 = read_from_hdf5("Five_qubits_state_entangle.h5","state")

In [ ]:
data=np.vstack((sep_file4,eng_file4 ))
labels=[-1 for i in range (6000)]+[1 for i in range (6000)]
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
dev6 = qml.device('default.qubit', wires=10)  
@qml.qnode(dev6)
def Classify_circuit(states,reduced_dimension,target_state_dimension,classify_dimesion,ansatz_type,weights):
    # total_register=list(range(target_state_dimension+reduced_dimension))
    state_register=list(range(target_state_dimension+reduced_dimension))
    first_register = list(range(reduced_dimension+classify_dimesion))
    classify_register=first_register[-classify_dimesion:]
    qml.StatePrep(state=states,wires=state_register)
    if ansatz_type == "StronglyEntanglingLayers":
        qml.StronglyEntanglingLayers(wires=classify_register, weights=weights)
    if ansatz_type == "hardware_efficient_ansatz":
        hardware_efficient_ansatz(wires=classify_register, weights=weights)
    if ansatz_type == "simplified_two_design":
        simplified_two_design(wires=classify_register, weights=weights)
    return qml.expval(qml.PauliZ(classify_register[-1])) 

In [ ]:
def square_loss(labels, predictions):
    out=0
    for index,element in enumerate(labels):
        out+=(labels[index] - predictions[index]) ** 2
    print(out)
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return out/len(labels)
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [ ]:
toltal_cost=[]
save_weights=[]
toltal_acc=[]
for n_w in [8,7,6,5]:
    for n_l in [3,4,5,6]:
        for types in ["hardware_efficient_ansatz","StronglyEntanglingLayers","simplified_two_design"]:
            def cost(weights ,X, Y):
                predictions = [Classify_circuit(states=x, reduced_dimension=2,target_state_dimension=5,
                                                classify_dimesion=n_w,ansatz_type=types,weights=weights)  for index ,x in enumerate (X)]
                return square_loss(Y, predictions)
            opt = qml.AdamOptimizer(0.04)
            if types == "StronglyEntanglingLayers":
                shape1 = qml.StronglyEntanglingLayers.shape(n_layers=n_l, n_wires=n_w)
                weights = np.random.random(size=shape1)
            if types == "hardware_efficient_ansatz":
                weights=generate_weights(num_wires=n_w,num_layers=n_l)
            if types == "simplified_two_design":
                weights=simplified_two_design_weights(num_layers=n_l,num_wires=n_w) 
            batch_size = 64
            costs=[]
            acc_list=[]
            # 训练模型
            for it in range(100):
                # Update the weights by one optimizer step, using only a limited batch of data
                batch_index = np.random.randint(0, len(X_train), (batch_size,))
                X_batch =[]
                Y_batch=[]
                for i in batch_index:
                    X_batch.append(X_train[i])
                    Y_batch.append(y_train[i])
                weights = opt.step(cost, weights, X=X_batch, Y=Y_batch)

                # Compute accuracy
                predictions = [np.sign(Classify_circuit(states=x, reduced_dimension=2,target_state_dimension=5,
                                                classify_dimesion=n_w,ansatz_type=types,weights=weights)) for index ,x in enumerate (X_batch)]
                # predictions = [np.sign(Classify_circuit(states=x, reduced_dimension=2,target_state_dimension=2,
                #                                 classify_dimesion=3,weights=weights)) for index ,x in enumerate (X_batch)]
                current_cost = cost(weights, X_batch, Y_batch)
                costs.append(current_cost)
                acc = accuracy(Y_batch, predictions)
                acc_list.append(acc)

                print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")
            toltal_acc.append(acc_list)
            toltal_cost.append(costs)
            save_weights.append(weights)